In [1]:
import numpy as np
import h5py
import os.path as op
import os 
import JONSWAP as J
import OmegaTheta as OT
import CreateSpectra as CS
import DtoF as DF
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# HOS: Initial Conditions

This script produces initial conditions in HDF5 format for the HOS model.

In [2]:
def write_hdf5(SimFolder, TestName, Nx, Ny, Lx, Ly, g, h, T, dtsave, saveflg, runsubid, Tramp, etar, phir):

    filename = SimFolder + 'initpars_' + str(TestName) + '.h5'
    
    if op.exists(filename): 
        os.remove(filename)
    
    with h5py.File(filename,'w') as file:

        file.create_dataset('Nx', data = Nx)
        file.create_dataset('Ny', data = Ny)
        file.create_dataset('Lx', data = Lx)
        file.create_dataset('Ly', data = Ly)
        file.create_dataset('g', data = g)
        file.create_dataset('KP', data = 10000)
        file.create_dataset('h', data = h)
        file.create_dataset('T', data = T)
        file.create_dataset('dtsave', data = dtsave)
        file.create_dataset('saveflg', data = saveflg)
        file.create_dataset('runsubid', data = runsubid)
        file.create_dataset('rampflg', data = 1)
        file.create_dataset('Tramp', data = Tramp)
        
    filename = SimFolder + 'initdata_' + str(TestName) + '.h5'
    
    if op.exists(filename): 
        os.remove(filename)
    
    with h5py.File(filename,'w') as file:
    
        file.create_dataset('eta0', data = etar)
        file.create_dataset('phi0', data = phir)

In [ ]:
def create_stokes():
    return 0

# Grid spacing in x direction
dx = Lx/Nx
x = np.asarray([x * dx for x in np.arange(0,Nx)])

#kx = [0:Nx/2 - Nx/2+1:-1]'*np.pi/Lx; 

dy = Ly/Ny;
y = np.asarray([y * dy for y in np.arange(0,Ny)])

#ky = [0:Ny/2 - Ny/2+1:-1]'*np.pi/Ly; 



# Wave amplitude
amp = 0.15

#kx = 32*2*pi/Lx;
#ky = 0*2*pi/Ly;
kx = float(1./2.)
ky = float(sqrt(3.)/2.)
#kx = 2*pi/Lx;
#ky = 0;


kap = np.sqrt(kx**2+ky**2)

ome = np.sqrt(kap)

# c = ome/kap + 0.5*amp^2*ome^5/kap;	
# Stokes_2d_eta = @(x,y,t)   amp*cos(kx*x + ky*y - kap*c*t) + 0.5*amp^2*abs(kap)*cos(2*kx*x + 2*ky*y - 2*kap*c*t);
# Stokes_2d_phi = @(x,y,t) c*amp*sin(kx*x + ky*y - kap*c*t) + 0.5*c*amp^2*abs(kap)*sin(2*kx*x + 2*ky*y - 2*kap*c*t);


N = 2048
L = np.ceil( np.sqrt( Lx**2 + Ly**2 )/(2. * np.pi / kap) ) * 2. *np.pi / kap

[ etas, phis, phis_xs, ws, c_stokes ] = StokesWave(amp, kap, L, N, 0)

hetas = fft(etas);
hphis = fft(phis);
hws = fft(ws);

kappa = [0:N/2 -N/2+1:-1]'*2*pi/L; 
sk = sign(kappa);
xis = (0:N-1).'*L/N;
xs = xis - real(ifft(( 1i*sk.*hetas )));


Nlarge = 32*N;
xis_large = (0:Nlarge-1).'*L/Nlarge;

hetas_pad = zeros(Nlarge,1);
hetas_pad(1:N/2+1) = hetas(1:N/2+1)*Nlarge/N; 
hetas_pad(end-N/2+2:end) = hetas(N/2+2:end)*Nlarge/N;
etas_int = real(ifft(hetas_pad));

hphis_pad = zeros(Nlarge,1);
hphis_pad(1:N/2+1) = hphis(1:N/2+1)*Nlarge/N; 
hphis_pad(end-N/2+2:end) = hphis(N/2+2:end)*Nlarge/N;
phis_int = real(ifft(hphis_pad));

hws_pad = zeros(Nlarge,1);
hws_pad(1:N/2+1) = hws(1:N/2+1)*Nlarge/N; 
hws_pad(end-N/2+2:end) = hws(N/2+2:end)*Nlarge/N;
ws_int = real(ifft(hws_pad));

hxs_pad = zeros(Nlarge,1);
hxs = fft(xs - xis);
hxs_pad(1:N/2+1) = hxs(1:N/2+1)*Nlarge/N; 
hxs_pad(end-N/2+2:end) = hxs(N/2+2:end)*Nlarge/N;
xs_int = real(ifft(hxs_pad)) + xis_large;


Stokes_eta_exact = @(xx) interp1([ xs_int  - L;  xs_int; xs_int  + L], [etas_int; etas_int; etas_int], xx, 'cubic');
Stokes_phi_exact = @(xx) interp1(xs_int, phis_int, xx, 'cubic');
Stokes_w_exact = @(xx) interp1(xs_int, ws_int, xx, 'cubic');

etasx = Stokes_eta_exact(xis);
phisx = Stokes_phi_exact(xis);
wsx = Stokes_w_exact(xis);

Stokes_2d_eta = @(x,y,t) Stokes_eta_exact( (kx*x + ky*y)/kap );
Stokes_2d_phi = @(x,y,t) Stokes_phi_exact( (kx*x + ky*y)/kap );
Stokes_2d_w   = @(x,y,t) Stokes_w_exact( (kx*x + ky*y)/kap );

eta0 = zeros(Nx,Ny);
phi0 = zeros(Nx,Ny);
w0 = zeros(Nx,Ny);

#for ix=1:Nx;
    for ny=1:Ny

        #x = dxi*(ix-1)
        y = dy*(ny-1);

        eta0(:,ny) = Stokes_2d_eta(x,y,0) + 0.1*cos(kx*x); #...
        #+ 0.001*cos(x/8) + 0.001*cos(x/4) + 0.001*cos(x/2) ...
        #+ 0.001*cos(x/8 + y/8) + 0.001*cos(x/4 + y/4) + 0.001*cos(x/4 + y/2);
        phi0(:,ny) = Stokes_2d_phi(x,y,0);
        w0(:,ny) = Stokes_2d_w(x,y,0);
        #eta0(ix,iy) = 0.1*sin(kx*dxi*(ix-1));
        #phi0(ix,iy) = 0.05/sqrt(sqrt(kx^2+ky^2))*cos(kx*dxi*(ix-1) + ky*dyi*(iy-1));
        #phi0(ix,iy) = cos(2*pi*dyi*(iy-1)/Ly);
        #phi0(ix,iy) = ix + Nx*(iy-1);
    
    end
#end


In [ ]:
# Test for the JONSWAP spectrum

g     = 9.8               #acceleration due to gravity
alpha = 1.0 #0.125

#-- Discretization: Number of cells in x and y direction
Nx = alpha*512           #fourier modes in x/y directions
Ny = alpha*512 

kp    = 0.033;                #for deep water
lmbda = 2*np.pi/kp;
Lx    = 16*lmbda              # Lx = 1*ceil(((epsilon^-2)*pi/kp)); Lx  = 16*lambda;4.*np.pi
Ly    = Lx                   # Ly = 4.*np.pi/sqrt(3.)

kx = 2*np.pi/Lx*np.arange(-Nx/2,Nx/2)
ky = 2*np.pi/Ly*np.arange(-Ny/2,Ny/2)
 
etar, phir = CS.create_jonswap(kx,ky)

fig = plt.figure(1)
#plt.contourf(kx,ky,etar)
plt.imshow(etar)
fig = plt.figure(2)
plt.contourf(kx,ky,phir)
plt.colorbar()
plt.show()

JONSWAP.py:47: RuntimeWarning: divide by zero encountered in power
  const = np.multiply(alpha_p*g**2,np.power(omega,-5))
JONSWAP.py:53: RuntimeWarning: divide by zero encountered in power
  exponent = np.multiply(-1.25,np.power(np.divide(omega,omega_p),-4))
DtoF.py:55: ComplexWarning: Casting complex values to real discards the imaginary part
  hetaF[ikx,iky] = 0.5*norm*hetaD[ikx,iky] + 0.5*norm*np.conj(hetaD[ikxm, ikym])
DtoF.py:56: ComplexWarning: Casting complex values to real discards the imaginary part
  hphiF[ikx,iky] = -1j*0.5*norm*alpha*hetaD[ikx,iky] + 1j*0.5*alpha*norm*np.conj(hetaD[ikxm, ikym])
DtoF.py:52: RuntimeWarning: invalid value encountered in double_scalars
  norm  = np.sqrt(0.5*np.sqrt(g)*(np.sqrt(np.tanh(kh)/k**3) + h*(1/np.sqrt(k*np.tanh(kh)))*sech(kh)**2))


In [ ]:
def create_gauss():
    return 0

In [ ]:
def create_stokes():
    return 0

In [ ]:
SimFolder = '/home/nicole/Development/hos/python/'
TestName  = 'test'

#-- Labels for the simulation run
saveflg  = 1
runsubid = 1

#-- Wave field Parameters 
g     = 9.8               #acceleration due to gravity
alpha = 1.0

#-- Discretization: Number of cells in x and y direction
Nx = alpha*512            #fourier modes in x/y directions
Ny = alpha*512 

#-- Number of grid cells
N = Nx*Ny


#kp      = 4*pi^2/g; %(Tp = 1sec)
kp      = 0.033;                #for deep water
lmbda  = 2*np.pi/kp;
omega_p = np.sqrt(g*kp);
epsilon = 0.12;
theta_p = 12;
h       = 1000;             #water depth... set high to induce deep water effects
sigma   = epsilon/2/kp;     #desired eta variance
hs      = 4*sigma;          #sig. wave height
#hs = 0.005;


#kpx = kp*np.cos(theta_p);
#kpy = kp*np.sin(theta_p);

#-- Length of the domain
Lx = 64*lmbda              # Lx = 1*ceil(((epsilon^-2)*pi/kp)); Lx  = 16*lambda;4.*np.pi
Ly  = Lx                   # Ly = 4.*np.pi/sqrt(3.)

            
#-- Wavenumber Domain
kx = 2*np.pi/Lx*np.arange(-Nx/2,Nx/2)
ky = 2*np.pi/Ly*np.arange(-Ny/2,Ny/2)

#KX, KY = np.meshgrid(kx,ky);
#KA       = np.abs(KX + 1j*KY);

#-- Time parameters
Tp     = 1                     #Tp = 2*pi/omega_p;

#-- Final simulation time
T = 150*Tp                     #T=100; 20; T= 2*floor((epsilon^(-2)))*Tp; #BF timescale
dtsave = 1.0

Tramp  = 200

value = "JONSWAP"

switch = {'JONSWAP': create_jonswap,
     'Gauss': create_gauss,
     'Stokes': create_stokes}

if value in switch:
     etar, phir = switch[value](kx,ky)
else:
     pass
    #default
        
plt.figure(1)
plt.contourf(kx,ky,etar)
plt.colorbar()
plt.show()

# Write information
write_hdf5(SimFolder, TestName, Nx, Ny, Lx, Ly, g, h, T, dtsave, saveflg, runsubid, Tramp, etar, phir)